# <font color='FF6F1F'>Case Gol</font>

## <font color='FF6F1F'>Respostas às perguntas de interesse</font>

# Tabela de conteúdo

[1. Informação básica](#1.-Informação-básica)

[2. Bibliotecas](#2.-Bibliotecas)

[3. Dados](#3.-Dados)

[4. Perguntas](#4.-Perguntas)

* [1. Faça um ranking para o número total de PAX por dia da semana.](#1.-Faça-um-ranking-para-o-número-total-de-PAX-por-dia-da-semana.)

* [2. Qual a correlação de sábado e domingo somados com o total de RPK?](#2.-Qual-a-correlação-de-sábado-e-domingo-somados-com-o-total-de-RPK?)

* [3. Qual a média de ‘Monetário’ por mês por Canal? E a mediana?](#3.-Qual-a-média-de-‘Monetário’-por-mês-por-Canal?-E-a-mediana?)

* [4. Crie um forecast de PAX por ‘Local de Venda’ para os próximos 15 dias a contar da última data de venda.](#4.-Crie-um-forecast-de-PAX-por-‘Local-de-Venda’-para-os-próximos-15-dias-a-contar-da-última-data-de-venda.)

* [5. Supondo que você precisa gerar um estudo para a área responsável, com base em qualquer modelo ou premissa, qual ‘Local de Venda’ você considera mais crítico. Por quê?](#5.-Supondo-que-você-precisa-gerar-um-estudo-para-a-área-responsável,-com-base-em-qualquer-modelo-ou-premissa,-qual-‘Local-de-Venda’-você-considera-mais-crítico.-Por-quê?)

* [6. Criar modelo relacionando o comportamento de venda com variáveis não apresentadas nos dados (Ex: PIB, Dolar, etc.)](#6.-Criar-modelo-relacionando-o-comportamento-de-venda-com-variáveis-não-apresentadas-nos-dados-(Ex:-PIB,-Dolar,-etc.))


## 1. Informação básica
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

**Fonte:** Gol

**Descrição das colunas:**

| Variável | Descrição | Tipo |  
|:---|:---|:---|
| Data Venda | Data (AAAA-MM-DD) | Data |
| Canal de Venda | Canais de venda. | Nominal |
| Local de Venda | Locais de venda. | Nominal |
| PAX | Total de passageiros. | Discreto |
| Monetário Vendido | Valores da venda. | Contínuo |
| RPK | Indicador diretamente relacionado com o número de PAX. | Discreto |

## 2. Bibliotecas
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [1]:
import pandas as pd # dataframe
import numpy as np # numeric
import seaborn as sns # graph
from matplotlib import pyplot as plt # plot

## 3. Dados
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [2]:
# suprime a notação científica
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
df = pd.read_excel('case_analytics-ds.xlsx', sheet_name='Dados')

In [4]:
df.sample(5)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK
608,2016-12-09,Telégrafo,Vast,65,2621.2220,3102
2213,2017-03-19,Porta a Porta,Arena,916,38324.3133,132375
1060,2017-01-06,TeleVenda,Vast,325,19295.1918,15576
1714,2017-02-16,Telégrafo,Mindscape,82,5210.7499,9507
685,2016-12-13,Telégrafo,Mindscape,70,2840.3579,9501


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2432 entries, 0 to 2431
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Data Venda         2432 non-null   datetime64[ns]
 1   Canal de Venda     2432 non-null   object        
 2   Local de Venda     2432 non-null   object        
 3   PAX                2432 non-null   int64         
 4   Monetário Vendido  2432 non-null   float64       
 5   RPK                2432 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 114.1+ KB


In [6]:
print(f'O dataframe possui {df.shape[0]} linhas e {df.shape[1]} colunas.')

O dataframe possui 2432 linhas e 6 colunas.


In [7]:
# Dados faltantes
total = df.isnull().sum().sort_values(ascending=False)
porcentagem = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
faltante = pd.concat([total, porcentagem], axis=1, keys=['Total', 'Porcentagem'])
faltante.head(45)

,Total,Porcentagem
RPK,0,0.0000
Monetário Vendido,0,0.0000
PAX,0,0.0000
Local de Venda,0,0.0000
Canal de Venda,0,0.0000
Data Venda,0,0.0000


* Não há dados faltantes.

In [8]:
# lista dos canais de venda
df['Canal de Venda'].unique().tolist()

['Telégrafo', 'TeleVenda', 'Porta a Porta']

In [9]:
# lista dos locais de venda
df['Local de Venda'].unique().tolist()

['Mindscape', 'Arena', 'Vast', 'Ellipsis']

In [10]:
df.describe()

,PAX,Monetário Vendido,RPK
count,2432.0000,2432.0000,2432.0000
mean,723.0029,24638.7405,66738.3894
std,735.9365,22424.8429,83479.0395
min,18.0000,1041.5444,1127.0000
25%,100.0000,6122.9205,11372.0000
50%,395.5000,17755.4217,33361.5000
75%,1160.2500,35535.3240,95116.7500
max,4705.0000,144890.7311,695824.0000


## 4. Perguntas
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

### 1. Faça um ranking para o número total de PAX por dia da semana.
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [11]:
# cópia do dataframe
df_ = df.copy()

In [12]:
# cria dia da semana
df_['dia_semana'] = df_['Data Venda'].dt.day_name()

In [13]:
# en --> pt
df_.loc[df_['dia_semana'] == 'Monday', 'dia_semana'] = 'segunda-feira'
df_.loc[df_['dia_semana'] == 'Tuesday', 'dia_semana'] = 'terça-feira'
df_.loc[df_['dia_semana'] == 'Wednesday', 'dia_semana'] = 'quarta-feira'
df_.loc[df_['dia_semana'] == 'Thursday', 'dia_semana'] = 'quinta-feira'
df_.loc[df_['dia_semana'] == 'Friday', 'dia_semana'] = 'sexta-feira'
df_.loc[df_['dia_semana'] == 'Saturday', 'dia_semana'] = 'sábado'
df_.loc[df_['dia_semana'] == 'Sunday', 'dia_semana'] = 'domingo'

In [14]:
df_.sample(5)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK,dia_semana
1486,2017-02-01,TeleVenda,Mindscape,172,10815.2092,19437,quarta-feira
1320,2017-01-22,TeleVenda,Ellipsis,1534,49885.1596,85311,domingo
318,2016-11-20,Porta a Porta,Ellipsis,686,22111.3051,37225,domingo
193,2016-11-13,TeleVenda,Mindscape,1653,64793.5112,206361,domingo
2307,2017-03-25,TeleVenda,Arena,2093,75574.9898,309972,sábado


In [15]:
df_.groupby(by='dia_semana')['PAX'].sum().sort_values(ascending=False)

dia_semana
quarta-feira     293025
quinta-feira     258488
sexta-feira      255625
sábado           253467
domingo          241620
segunda-feira    236316
terça-feira      219802
Name: PAX, dtype: int64

In [16]:
pax_dia = df_.groupby('dia_semana').agg({'PAX': ['sum']})
pax_dia

,PAX
,sum
dia_semana,
domingo,241620
quarta-feira,293025
quinta-feira,258488
segunda-feira,236316
sexta-feira,255625
sábado,253467
terça-feira,219802


* O dia com maior número de passageiros, em todo período, é quarta-feira.

### 2. Qual a correlação de sábado e domingo somados com o total de RPK?
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [17]:
# dia da semana: segunda-feira == 0; domingo == 6
df_['dia_semana_num'] = df_['Data Venda'].dt.dayofweek

In [18]:
df_.sample(3)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK,dia_semana,dia_semana_num
306,2016-11-20,Telégrafo,Ellipsis,41,2750.7479,2189,domingo,6
1972,2017-03-04,Porta a Porta,Ellipsis,905,23825.9166,49737,sábado,5
662,2016-12-12,Telégrafo,Ellipsis,58,2572.3585,3407,segunda-feira,0


In [19]:
df_.groupby('dia_semana_num').agg({'RPK': ['sum']})

,RPK
,sum
dia_semana_num,
0,23275908
1,20366840
2,26263705
3,23501102
4,23309936
5,23201577
6,22388695


In [20]:
sabado_RPK = df_[['dia_semana_num','RPK']].query('`dia_semana_num` == 5').sum()
sabado_RPK

dia_semana_num        1760
RPK               23201577
dtype: int64

In [21]:
domingo_RPK = df_[['dia_semana_num','RPK']].query('`dia_semana_num` == 6').sum()
domingo_RPK

dia_semana_num        2016
RPK               22388695
dtype: int64

In [22]:
total_RPK = df_['RPK'].sum()
total_RPK

162307763

In [23]:
RPK_sab_dom = (sabado_RPK[1] + domingo_RPK[1]) / total_RPK

In [24]:
print(f'Sábado e domingo representam {round(RPK_sab_dom * 100, 2)}% do total RPK.') 

Sábado e domingo representam 28.09% do total RPK.


### 3. Qual a média de ‘Monetário’ por mês por Canal? E a mediana?
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [25]:
# coluna 'mês'
df_['mês'] = df['Data Venda'].dt.month

In [26]:
df_.sample(5)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK,dia_semana,dia_semana_num,mês
2161,2017-03-16,TeleVenda,Arena,363,19359.7233,53293,quinta-feira,3,3
557,2016-12-05,TeleVenda,Ellipsis,104,6071.4518,6534,segunda-feira,0,12
722,2016-12-16,TeleVenda,Ellipsis,1274,38359.1675,70586,sexta-feira,4,12
1562,2017-02-06,Porta a Porta,Ellipsis,2284,40497.7827,132854,segunda-feira,0,2
599,2016-12-08,Telégrafo,Mindscape,94,7989.6182,11046,quinta-feira,3,12


In [27]:
df_.groupby(['Canal de Venda', 'mês']).agg({'Monetário Vendido': ['mean', 'median']})

Monetário Vendido           
                                mean     median
Canal de Venda mês                             
Porta a Porta  1          31320.5695 28823.0253
               2          31710.1489 29711.5801
               3          31055.9398 30488.1910
               4          33319.4963 32421.6094
               11         34638.0718 30262.9780
               12         28615.4525 25527.5193
TeleVenda      1          32868.8814 19085.1232
               2          32208.4458 18926.6902
               3          32132.2467 18666.8800
               4          35795.7819 28091.8856
               11         30963.4593 18757.7078
               12         28045.7215 17567.6009
Telégrafo      1           4234.0226  3782.2489
               2           4307.6263  4045.8931
               3           4568.1485  4345.5596
               4           3778.5918  3458.8847
               11          4614.3101  4029.1321
               12          5212.6481  4227.5779

### 4. Crie um forecast de PAX por ‘Local de Venda’ para os próximos 15 dias a contar da última data de venda.
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [ ]:
# https://github.com/PracticalTimeSeriesAnalysis/BookRepo
# https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/
# https://stackoverflow.com/questions/22770352/auto-arima-equivalent-for-python

In [ ]:
# ARIMA example
from statsmodels.tsa.arima.model import ARIMA
from random import random
# contrived dataset
# data = [x + random() for x in range(1, 100)]
# fit model
model = ARIMA(df_, order=(1, 1, 1))
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(df_), len(df_), typ='levels')
print(yhat)

In [ ]:
df_.info()

In [ ]:
# transformar Local de Venda em número
# eliminar object columns: dia_semana
# criar novo dataset

In [ ]:
df_['Canal de Venda'].unique().tolist()

In [ ]:
canais = {'Telégrafo': 1, 'TeleVenda': 2, 'Porta a Porta': 3}

### 5. Supondo que você precisa gerar um estudo para a área responsável, com base em qualquer modelo ou premissa, qual ‘Local de Venda’ você considera mais crítico. Por quê?
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [ ]:
# Monerário Vendido por Canal de Venda - período total
df_.groupby('Local de Venda').agg({'Monetário Vendido': ['mean', 'median', 'sum']})

In [ ]:
sns.barplot(x='Local de Venda', y='Monetário Vendido', data=df_)

* O local de venda mais crítico é Ellipsis, possui um total de vendas no período mais baixo comparando-o com os outros locais.

### 6. Criar modelo relacionando o comportamento de venda com variáveis não apresentadas nos dados (Ex: PIB, Dolar, etc.)
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [ ]:
# cria ano
df_['ano'] = df['Data Venda'].dt.year

In [ ]:
# lista anos 
df_['ano'].unique().tolist()

In [ ]:
# lista meses 
df_['mês'].unique().tolist()

In [ ]:
# https://economia.acspservicos.com.br/indicadores_iegv/iegv_dolar.html
## dict = {mês: dólar}
dolar_2016 = {1: 4.054, 2: 3.978, 3: 3.694, 4: 3.551, 11: 3.339, 12: 3.351}
dolar_2017 = {1: 3.197, 2: 3.103, 3: 3.127, 4: 3.140, 11: 3.257, 12: 3.297} 

# cria coluna dólar
for d in df_['ano']:
    if df_['ano'][d] == 2016:
        df_['dólar'] = df_['mês'].map(dolar_2016)
    else:
        df_['dólar'] = df_['mês'].map(dolar_2017)

In [ ]:
# https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas
## dict = {mês: ipca(%)}
ipca_2016 = {1: 1.27, 2: 0.90, 3: 0.43, 4: 0.61, 11: 0.18, 12: 0.30}
ipca_2017 = {1: 0.38, 2: 0.33, 3: 0.25, 4: 0.14, 11: 0.28, 12: 0.44}

# cria coluna ipca
for i in df_['ano']:
    if df_['ano'][i] == 2016:
        df_['ipca'] = df_['mês'].map(ipca_2016)
    else:
        df_['ipca'] = df_['mês'].map(ipca_2017)

In [ ]:
df_.sample(3)

In [ ]:
df_[['Monetário Vendido','mês', 'ano', 'dólar', 'ipca']].describe()

In [ ]:
corr = df_[['Monetário Vendido','mês', 'ano', 'dólar', 'ipca']].corr()
corr

In [ ]:
sns.clustermap(corr, cmap='Oranges', annot = True, figsize=(6, 6))

In [ ]:
Mais do que o resultado, iremos avaliar o raciocínio e a organização de trabalho

Algumas considerações:

*PAX é o total de passageiros. RPK é um indicador diretamente relacionada com o número de PAX.
*Não se atenha às grandezas. Os dados são fictícios. 😉
*Envie todo o material que produzir (códigos, tabelas e outros arquivos) com o detalhamento de cada um. Se possível, comente nos códigos.
*Para a apresentação, use PowerPoint ou qualquer outra ferramenta de DataViz que julgar pertinente.
